Use [RISE](https://github.com/damianavila/RISE) to display the slides.

```bash
pip install RISE
```

### Distributiivne semantika - proovime ise järgi

In [1]:
# Impordime vajalikud moodulid
import csv
from gensim.models import Word2Vec, KeyedVectors
from estnltk import Text
from collections import Counter

In [2]:
# model from here http://193.40.33.66/pretrained/cbow_100_5_10_20.zip
# documentation here https://github.com/eleriaedmaa/embeddings

# mudelifaili tee vaja asendada sellega, kuhu mudeli alla laadisite oma arvutis
model = KeyedVectors.load_word2vec_format("../../../AREA1/cbow_100_5_10_20/ettenten.txt.word_vectors", binary=False)

INFO:utils_any2vec.py:274: loading projection weights from ../../../AREA1/cbow_100_5_10_20/ettenten.txt.word_vectors
INFO:utils_any2vec.py:297: loaded (470688, 100) matrix from ../../../AREA1/cbow_100_5_10_20/ettenten.txt.word_vectors


In [3]:
most_similar = model.most_similar(positive=['kurk', 'porgand'], topn=100)

INFO:keyedvectors.py:1353: precomputing L2-norms of word weight vectors


In [4]:
most_similar[:20]

[('tomat', 0.8834239840507507),
 ('õun', 0.8118515014648438),
 ('arbuus', 0.8091526031494141),
 ('kartul', 0.8073391914367676),
 ('sibul', 0.8037582635879517),
 ('maasikas', 0.8013646602630615),
 ('küüslauk', 0.7977540493011475),
 ('hernes', 0.7962828874588013),
 ('värske_kurk', 0.7874271869659424),
 ('kapsas', 0.7873393297195435),
 ('juurikas', 0.7767490148544312),
 ('kaalikas', 0.7753903865814209),
 ('paprika', 0.7706528902053833),
 ('mädarõigas', 0.7664803266525269),
 ('banaan', 0.7660707235336304),
 ('toores_porgand', 0.7492185831069946),
 ('kõrvits', 0.7491439580917358),
 ('purk_marineerima', 0.7484642863273621),
 ('tomat_salat', 0.7462498545646667),
 ('vaarikas', 0.7438136339187622)]

In [5]:
most_similar = model.most_similar(positive=['müüja', 'dirigent', 'ehitaja'], topn=100)

In [6]:
most_similar[:20]

[('töömees', 0.7112044095993042),
 ('ehitusfirma', 0.6680212616920471),
 ('ehitusettevõtja', 0.6548689603805542),
 ('klienditeenindaja', 0.6475248336791992),
 ('peatöövõtja', 0.6439967155456543),
 ('ehitusmees', 0.6306071877479553),
 ('töödejuhataja', 0.6255625486373901),
 ('alltöövõtja', 0.6250219345092773),
 ('maaler', 0.622079610824585),
 ('haljastusfirma', 0.6190921068191528),
 ('ehitusjuht', 0.613804817199707),
 ('teenindaja', 0.611613392829895),
 ('ehitusettevõte', 0.6074643135070801),
 ('müügimees', 0.6037381887435913),
 ('kaido_fridolin', 0.6015191674232483),
 ('õmbleja', 0.6008065938949585),
 ('maakler', 0.6000090837478638),
 ('vahetusevanem', 0.5988837480545044),
 ('tellija', 0.596854567527771),
 ('paigaldaja', 0.5958028435707092)]

### Oma nimeüksuste lisamine

1) Leiame võimalikult suure hulga näiteid vastavast nimeüksusest
 
2) Märgendame ümber olemasoleva korpuse - anname seal esinevatele vastavatele nimeüksustele soovitud märgendid

3) Treenime nimeüksuste tuvastaja ümbermärgendatud korpuse peal

4) Märgendame oma nimeüksusi

In [7]:
from estnltk.taggers.estner.ner_trainer import NerTrainer
from estnltk.taggers.estner.refac.ner import ModelStorageUtil
from estnltk.core import DEFAULT_PY3_NER_MODEL_DIR
from estnltk.taggers import NerTagger

ner_tagger = NerTagger()
model_dir=DEFAULT_PY3_NER_MODEL_DIR
modelUtil = ModelStorageUtil(model_dir)
nersettings = modelUtil.load_settings()
trainer = NerTrainer(nersettings)

In [8]:
# See on meie treeningkorpus - peab olema morfoloogiliselt märgendatud
text = Text('Eesti president käis Euroopas.').tag_layer()

In [9]:
text.words

Layer(name='words', attributes=('normalized_form',), spans=SL[Span('Eesti', [{'normalized_form': None}]),
Span('president', [{'normalized_form': None}]),
Span('käis', [{'normalized_form': None}]),
Span('Euroopas', [{'normalized_form': None}]),
Span('.', [{'normalized_form': None}])])

In [10]:
# Igale sõnale peab vastama märgend; iga lause kohta üks list
labels = [['B-LOC','B-VOC','O','B-LOC', 'O']]

In [ ]:
# Treenime, kasutades treeningkorpust ja märgendeid;
# salvestame tulemuse kausta 'test'
trainer.train(text, labels, 'test')

In [12]:
# See on meie testkorpus - ei kattu treeningkorpusega
text2 = Text('Läti president elab Riias.').tag_layer()

In [13]:
# Standardne NER-tagger leiab asukohad
ner_tagger.tag(text2)

Text(text='Läti president elab Riias.')

In [14]:
text2.ner

Layer(name='ner', attributes=('nertag',), spans=SL[EnvelopingSpan(['Läti'], [{'nertag': 'LOC'}]),
EnvelopingSpan(['Riias'], [{'nertag': 'LOC'}])])

In [15]:
# Loome uue NER-taggeri oma loodud 'test' mudeli põhjal
# Anname väljundkihile nimeks ner2, 
# sest ner kiht on juba meie tekstil olemas
ner_tagger2 = NerTagger(model_dir = 'test', output_layer = 'ner2')

In [16]:
ner_tagger2.tag(text2)

Text(text='Läti president elab Riias.')

In [17]:
text2.ner

Layer(name='ner', attributes=('nertag',), spans=SL[EnvelopingSpan(['Läti'], [{'nertag': 'LOC'}]),
EnvelopingSpan(['Riias'], [{'nertag': 'LOC'}])])

In [18]:
text2.ner2

Layer(name='ner2', attributes=('nertag',), spans=SL[EnvelopingSpan(['Läti'], [{'nertag': 'LOC'}]),
EnvelopingSpan(['president'], [{'nertag': 'VOC'}]),
EnvelopingSpan(['Riias'], [{'nertag': 'LOC'}])])

In [19]:
text3 = Text('Läti peaminister elab ka Riias.').tag_layer()
ner_tagger2.tag(text3)
text3.ner2

Layer(name='ner2', attributes=('nertag',), spans=SL[EnvelopingSpan(['Läti'], [{'nertag': 'LOC'}]),
EnvelopingSpan(['peaminister'], [{'nertag': 'VOC'}]),
EnvelopingSpan(['Riias'], [{'nertag': 'LOC'}])])

<img src="pildid/kokkuvõte1.png">

<img src="pildid/kokuvõte2.png">

<img src="pildid/kokkuvõte.png">

<img src="pildid/lõpp.png">